In [1]:
import pandas as pd

In [2]:
# Load CSV paragraph 
# Ganti dengan yang baru 'labeled_man_partial.csv'
df_data_paragraph = pd.read_csv('labeled_man_final.csv')

# Load CSV other
df_data_table = pd.read_csv('material_book_2_other.csv')

In [3]:
df_data_paragraph.drop(columns='Unnamed: 0', inplace=True)
df_data_paragraph.head()

,ID,TYPE,VALUE,style_type,style_left,style_hanging,numPr_type,numPr_ilvl,numPr_id,label_id,label
0,0,CT_Empty,[w:drawing],[],[],[],[],[],[],1,Announcement
1,1,text,Kata Pengantar,[],[],[],[],[],[],1,Announcement
2,2,text,"Tak terasa, kita sudah akan kembali menapaki M...",[],[],[],[],[],[],1,Announcement
3,3,text,Masa Adven-Natal 2020 ini sengaja mengambil te...,[],[],[],[],[],[],1,Announcement
4,4,text,Buku ini dapat terwujud sedemikian rupa karena...,[],[],[],[],[],[],1,Announcement


In [ ]:
# Check variation data paragraph
# If we see from atribute "style_left", there are a lot of variation and it's count. 
# There are top 10 of it's variation
df_data_paragraph.style_left.value_counts().nlargest(10)

In [ ]:
# Check variation data paragraph
# Combination of "style_left" and "style_hanging"
df_data_paragraph.groupby(['style_left', 'style_hanging']).size().nlargest(10)

In [ ]:
# How many ID


In [ ]:
# Add columns ID and its value
# df_data_paragraph = df_data_paragraph.drop(columns='ID')
# df_data_paragraph.head()

In [ ]:
df_data_paragraph.insert(0, 'ID', range(0, 2729))
df_data_paragraph.head()
# df_data_paragraph.shape

### Add Label Columns

In [ ]:
# Add Columns Label and label_id
df_data_paragraph['label_id'] = 0
df_data_paragraph['label'] = None
df_data_paragraph.head()

In [ ]:
# Automation labeling using format (assumptions)
# Theres no style using sermon label
df_data_paragraph.loc[((df_data_paragraph['style_left'] == '[]') & (df_data_paragraph['style_hanging'] == '[]')
                      & (df_data_paragraph['numPr_type'] == '[]') & (df_data_paragraph['numPr_ilvl'] == '[]')), 'label_id'] = 1

df_data_paragraph.head()

In [ ]:
df_data_paragraph.loc[(df_data_paragraph['label_id'] == 1)]

In [ ]:
# Give value to label
# 1: Announcement; 2: Prayer; 3: Sermon; 4: Worship
df_data_paragraph.loc[(df_data_paragraph['label_id'] == 1), 'label'] = 'Announcement'
df_data_paragraph.head()

In [ ]:
# Bagian Pertama <Bastian> H.73
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Sukaria yang diberikan Tuhan dalam menyambut")]

In [ ]:
df_data_paragraph.loc[(df_data_paragraph['ID'] == 370), 'label_id'] = 3
df_data_paragraph.loc[(df_data_paragraph['ID'] == 370), 'label'] = 'Sermon'

In [ ]:
df_data_paragraph.loc[(df_data_paragraph['ID'] == 370)]

### Sermon Labeling
Use patterns to distinguish between announcement & Sermon
- Sermon always started with the title "Khotbah" in documents
- Ended with terms '[AWi]' clue: use regex

In [ ]:
# Marker of start shermon
marker_start_shermon = df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("KHOTBAH")]

In [ ]:
# Marker of End Shermon
import re
marker_end_shermon = df_data_paragraph.loc[df_data_paragraph.VALUE.apply(lambda x: bool(re.match("^\[\w{1,3}\s*\w{1,3}]$", x)))]

In [ ]:
lst_sermon_ID = []
for id_sermon_start in marker_start_shermon.ID:
    lst_start_end_id = []
    lst_start_end_id.append(id_sermon_start)
    for id_sermon_end in marker_end_shermon.ID:
        if id_sermon_end > id_sermon_start:
            lst_start_end_id.append(id_sermon_end)
            break
    lst_sermon_ID.append(lst_start_end_id)
lst_sermon_ID

In [ ]:
# Mengubah Label id  menjadi 3 <Sermon> dari list_id_sermon yang sudah tersedia
for ids in lst_sermon_ID:
    for i in range(ids[0],ids[1]):
        df_data_paragraph.loc[(df_data_paragraph['ID'] == i), 'label_id'] = 3
df_data_paragraph.loc[(df_data_paragraph['label_id'] == 3)]

In [ ]:
# Mengubah label menjadi "Sermon" (yang memiliki label_id = 3)
df_data_paragraph.loc[(df_data_paragraph['label_id'] == 3), 'label'] = 'Sermon'
df_data_paragraph.loc[(df_data_paragraph['label_id'] == 3)]

### Using format style to label Worship
Assumption : Labeling all row to worship for spesific style and then evaluate

In [ ]:
df_data_paragraph.loc[(((df_data_paragraph['style_hanging'] == '[]') | (df_data_paragraph['style_hanging'] == '425')) &
                       ((df_data_paragraph['style_left'] == '426') | (df_data_paragraph['style_left'] == '851') |
                       (df_data_paragraph['style_left'] == '709')))]

In [ ]:
# df_data_paragraph.loc[(df_data_paragraph['ID'] == 2550)]
df_data_paragraph.loc[(df_data_paragraph['style_left'] == '709')]

In [ ]:
df_data_paragraph.loc[(((df_data_paragraph['style_hanging'] == '[]') | (df_data_paragraph['style_hanging'] == '425')) &
                       ((df_data_paragraph['style_left'] == '426') | (df_data_paragraph['style_left'] == '851') |
                       (df_data_paragraph['style_left'] == '709'))), 'label_id'] = 4
df_data_paragraph.loc[(df_data_paragraph['label_id'] == 4), 'label'] = 'Worship'
df_data_paragraph.loc[(df_data_paragraph['label_id'] == 4)]

### Prayer Labeling
Using format or regex (semi automation)

In [ ]:
df_data_paragraph.shape

### Pengecekan Manual 
Untuk Labeling yang salah atau belum dilabeli

In [4]:
inp = "Saya Mau Ikut Yesus"
df_data_paragraph.loc[(df_data_paragraph['VALUE'].str.contains(inp))]

,ID,TYPE,VALUE,style_type,style_left,style_hanging,numPr_type,numPr_ilvl,numPr_id,label_id,label
2673,2673,text,"NYANYIAN KJ 375 ""Saya Mau Ikut Yesus"" (Dinyany...",CT_Ind,284,284,numPr,0,64,4,Worship


In [8]:
df_data_paragraph.loc[(df_data_paragraph.ID == 2228)]

,ID,TYPE,VALUE,style_type,style_left,style_hanging,numPr_type,numPr_ilvl,numPr_id,label_id,label
2228,2228,text,Umat:\tSYUKUR KEPADA ALLAH!,CT_Ind,851,851,[],[],[],2,Prayer


In [ ]:
# 522 - 531 = Sermon <blm dilabeli>
# 2380 - 2392 = sermon? <Pengenaan Bahan kaum muda> <Hal 256-257>
# 2456 - 2465 = sermon? <Mendengarkan Firman Aduyuswa> <Hal 266-268>
# 2619 - 2626 = sermon <renungan> <Hal 282-284>
# 2662 - 2667 = sermon <renungan> <PD 4> <Hal 287-289>
for id_not_labeled in range(2662, 2668):
    df_data_paragraph.loc[(df_data_paragraph['ID'] == id_not_labeled), 'label_id'] = 3
df_data_paragraph.loc[(df_data_paragraph['label_id'] == 3), 'label'] = 'Sermon'

In [ ]:
###### 1400 - 1412 = Worship <blm dilabeli>
# 1425 - 1434 = worship
# 1455 - 1468 = worship <Hal 192>
# 1478 - 1494 = worship <Hal 193>
# 1501 - 1517 = worship <Hal 194>
# 1525 - 1534 = worship <hal 195>
# 1563 - 1571 = worship <Hal 197>
# 1576 - 1579 = worhsip <Hal 198>
# 1623 - 1628 = worship <Hal 201>
# 1635 - 1641 = worship <Hal 201>
# 1668 - 1681 = worship <Hal 203>
# 1692 - 1698 = worship <Hal 204>
# 1709 - 1717 = worship <Hal 205>
# 1728 - 1747 = worship <Hal 206>
# 1756 - 1768 = worship <Hal 207>
# 1774 - 1782 = worship <Hal 208>
# 1810 - 1826 = worship <Hal 209-210>
# 1830 - 1847 = worship <Hal 210>
# 1868 - 1872 = worship <Hal 213>
# 1897 - 1907 = worship <Hal 215>
# 1911 - 1920 = worship <Hal 215-216>
# 1926 - 1935 = worship <Hal 216>
# 1961 - 1964 = worship <Hal 217>
# 1972 - 1973 = worship <Hal 219>
# 1977 - 1985 = worship <Hal 219>
# 2000 - 2010 = worship <Hal 221-222> <Nyanyian Prosesi>
# 2019 - 2024 = worhsip <Hal 222-223> <Nyanyian Umat>
# 2029 - 2034 = worship <Hal 224>
# 2042 - 2048 = worship <Hal 225>
# 2050 - 2054 = worship <Hal 225>
# 2093 - 2099 = worship <Hal 228-229>
# 2121 - 2127 = worship <Hal 231>
# 2137 - 2155 = worship <Hal 232-233>
# 2168 - 2174 = worship <Hal 233>
# 2180 - 2184 = worship <Hal 234>
# 2192 - 2197 = worship <Hal 234-235>
# 2203 - 2209 = worship <Hal 235>
# 2212 - 2221 = worship <Hal 235-236>
# 2285 - 2299 = worship <Hal 241-242>
# 2587 - 2596 = worship <PD 3> <Hal 281>
# 2607 - 2616 = worship <PD 3> <Hal 282>
# 2628 - 2635 = worship <PD 3> <Fajar yang baru sudah rekah> <hal 285>
# 2640 - 2644 = worship <PD 3> <Fajar yang baru sudah rekah> <hal 285>
# 2649 - 2651 = worship <PD 4> <Hal 287>
# 2653 - 2659 = worship <PD 4> <Aku cinta Yesus 3 bahasa> <Hal 287>
# 2668 - 2669 = worship <PD 4> <Hal 289>
# 2673 - 2674 = worship <Hal 290>
for id_not_labeled in range(2673,2675):
    df_data_paragraph.loc[(df_data_paragraph['ID'] == id_not_labeled), 'label_id'] = 4
df_data_paragraph.loc[(df_data_paragraph['label_id'] == 4), 'label'] = 'Worship'

In [ ]:
# 1414 - 1423 = Prayer <blm dilabeli>
# 1450 - 1454 = prayer? 
# 1469 - 1474 = prayer? <Votum dan salam> <Hal 192>
# 1495 - 1500 = prayer <Hal 194>
# 1580 - 1588 = prayer <doa berkat> <Hal 198>
# 1611 - 1622 = prayer <pengakuan dosa> <Hal 200-201>
# 1683 - 1691 = prayer <Doa persembahan, pengutusan dan berkat> <Hal 203>
# 1699 - 1701 = prayer <Hal 204>
# 1718 - 1723 = prayer? <Votum dan salam> <Hal 205>
# 1750 - 1755 = prayer <Hal 207>
# 1873 - 1877 = prayer? <Votum> <Hal 214>
# 1880 - 1894 = prayer? <Puisi / poem> <Hal 214>
# 2012 - 2015 = prayer? <Votum> <Hal 222>
# 2025 - 2027 = prayer <Doa pengakuan dosa> <Hal 223>
# 2078 - 2088 = prayer <Doa Syafaat> <Hal 227>
# 2103 - 2107 = prayer? <Pengutusan dan berkat> <Hal 230>
# 2128 - 2133 = prayer? <Votum dan salam> <Hal 231-232>
# 2156 - 2166 = prayer <pengakuan dosa> <Hal 233>
# 2223 - 2228 = prayer? <Pengutusan> <Hal 236>
for id_not_labeled in range(2223,2229):
    df_data_paragraph.loc[(df_data_paragraph['ID'] == id_not_labeled), 'label_id'] = 2
df_data_paragraph.loc[(df_data_paragraph['label_id'] == 2), 'label'] = 'Prayer'

In [ ]:
# 1435 - 1449 = Announcement
# 1476 - 1477 = Announcement ? <Kata pembuka>
# 1518 - 1524 = Announcement <Berita Anugerah> <Hal 195>
# 1541 - 1562 = Announcement <Pelayanan Firman, pembacaan alkitab, pengakuan iman> <Hal 196-197>
# 1573 - 1575 = Announcement <Pengutusan>? <Hal 197-198>
# 1595 - 1604 = Announcement <Panggilan Beribadah> <Hal 199-200>
# 1630 - 1634 = Announcement <Berita Anugerah> <Hal 201>
# 1642 - 1667 = Announcement <Pelayanan Firman, pembacaan alkitab, pengakuan iman> <Hal 201-203>
# 1705 - 1708 = Announcement <Persiapan, mengajak berdiri> <Hal 205>
# 1724 - 1727 = Announcment <umat duduk, pendengarkan bunyi alam> <Hal 206>
# 1748 - 1749 = <Hal 207>
# 1769 - 1773 = <Berita Anugerah><Hal 208>
# 1783 - 1809 = Announcement <Pelayanan Firman, pembacaan alkitab, pengakuan iman> <Hal 208-209>
# 1827 - 1829 = <Umat berdiri, doa persembahan, pengutusan> <Hal 210>
# 1848 - 1855 = <Pengutusan dan berkat> <Hal 210-211>
# 1859 - 1867 = <Persiapan, Panggilan beribadah dan nyanyian> <Hal 213>
for id_not_labeled in range(1859,1868):
    df_data_paragraph.loc[(df_data_paragraph['ID'] == id_not_labeled), 'label_id'] = 1
df_data_paragraph.loc[(df_data_paragraph['label_id'] == 1), 'label'] = 'Announcement'

In [ ]:
# 1424 = Announcement <blm dilabeli>
# 2005 = <Prosesi>
df_data_paragraph.loc[(df_data_paragraph['ID'] == 2005), 'label_id'] = 1
df_data_paragraph.loc[(df_data_paragraph['label_id'] == 1), 'label'] = 'Announcement'

In [ ]:
df_data_paragraph.loc[(df_data_paragraph['ID'] == 1589), 'label_id'] = 2
df_data_paragraph.loc[(df_data_paragraph['label_id'] == 2), 'label'] = 'Prayer'

In [ ]:
# Ubah semua label 0 menjadi announcement <karena prayer, sermon dan worship sudah dilabeli>
df_data_paragraph.loc[(df_data_paragraph['label_id'] == 0), 'label_id'] = 1
df_data_paragraph.loc[(df_data_paragraph['label_id'] == 1), 'label'] = 'Announcement'

In [ ]:
# Final Save
df_data_paragraph.to_csv('labeled_man_final.csv')

### Find Verses in the Text
using regex

In [ ]:
verse_patterns = r'^([A-Z]).*\s\d*[:](\d*)|(\d*[-]\d*)'

In [ ]:
verse_matched = df_data_paragraph.loc[df_data_paragraph.VALUE.
                                      apply(lambda x: bool(re.match('^([A-Z][^KJNKB]).*\s{1}\d*[:](\d*)|(\d*[-]\d*)', x)))]
verse_matched

In [ ]:
# Add new columns isVerse to know that there is a verse in the text?
df_data_paragraph['isVerse'] = False
df_data_paragraph.head()

In [ ]:
verse_matched['isVerse'] = True
verse_matched

In [ ]:
df_data_paragraph.to_csv('data_1_label.csv')

## Search format of announcement in text

In [ ]:
# Heading 1
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Kata Pengantar")]

In [ ]:
# Paragraph
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Tak terasa, kita sudah akan kembali menapaki Masa Adven-Natal 2020. Kita bersyukur meski di tengah pandemi covid-19 seperti ini, kita masih diperkenan Tuhan untuk memersiapkan umat-Nya merayakan Adven-Natal. Kiranya buku ini membantu dan atau memberi inspirasi Ibu/Bapak/Saudara sehingga semarak dan sukacita Adven-Natal tetap dapat kita rasakan di tengah pandemi.")]

In [ ]:
# Paragraph
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("mengucapkan terima kasih yang sebesar-besarnya kepada:")]

In [ ]:
# Points
# asdasdasdasdasdasdasdasdasdasd
#   1. Point-1
#   2. Point-2
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Andreas")]

In [ ]:
# Independent Sentence

# asdasdasdasdasdasd(Example Paragraph)
# 
# Independent Sentence
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains(". Murtini Hehanussa")]

In [ ]:
# List of contents
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Masa Adven 1")]

In [ ]:
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Hal tersebut bukan hendak mengajak umat untuk menyerang agama lain, namun ")]

In [ ]:
# Heading 2
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Firman Allah Menjadi Manusia?")]

In [ ]:
# Bahan Kotbah awal
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Penting untuk selalu menempatkan tema khotbah ibadah Minggu dalam bingkai")]

In [ ]:
# Bahan Kotbah dua
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Bagian ini berisi pengakuan dan permohonan. ")]

In [ ]:
# Points pada khotbah dua
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Kesadaran jati diri manusia di hadapan Pencipta yang membawa pada sikap hormat dan takut ")]

In [ ]:
# Sermon are always placed after "Elongated Sermons"

In [ ]:
# PL Liturgi #1 ( There is "PL" in value )
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Jemaat yang dikasihi Tuhan. Bersama gereja ")]

In [ ]:
# PL Liturgi #2 ( There is "PL" in value )
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Pertolongan kita adalah di dalam nama Tuhan ")]

## Search format of worship lyrics in text

In [ ]:
# Worshop marker #1
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("(Menyanyikan KJ 76:1-2)")]

In [ ]:
# Worship Tittle #1
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Kau Yang Lama Dinantikan")]

In [ ]:
# Worship lyrics #1.1
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Kau yang lama dinantikan")]

In [ ]:
# Worship lyrics #1.2
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Raja mulia, Kau ")]

In [ ]:
# Worship title #2
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Hai Orang Yang Beriman")]

In [ ]:
# Worship marker #2

In [ ]:
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("(Menyanyikan NKB 52:1,3)")]

In [ ]:
# Worship title #2
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Hai Orang Yang Beriman")]

In [ ]:
# Worship lyrics #2
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Hai orang yang beriman, tetap waspadalah!")]

In [ ]:
# Worship marker #3
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("(Menyanyikan KJ 29:1, dengan penyesuaian)")]

In [ ]:
# Worship title #3
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Di Hadapan Tuhan Yesus")]

In [ ]:
# Worship lyrics #3
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("dapan Tuhan Yesus betapa hina diriku")]

In [ ]:
# Worship marker #4
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Nyanyian Syukur")]

In [ ]:
# Worship title #4
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("(Menyanyikan PKJ 202)")]

In [ ]:
# Worship lyrics #4
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Yesusku Jurus")]

In [ ]:
# Worship marker #5
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Cip. Pdt. Juswantori Ichwan/Wesley Tulus")]

In [ ]:
# Worship lyric #5
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Bapa kami yang ada di sorg")]

In [ ]:
# Worship lyric #5
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Persembahkan dirimu untuk Tuhan ")]

## Search format of Prayer in text

In [ ]:
# Prayer marker #1
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Pengakuan Dosa")]

In [ ]:
# Prayer content #1
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Siapakah kami ini ya Tuhan sehingga Kau anggap ")]

In [ ]:
# Prayer content #2
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Siapakah kami ini ya Tuhan sehingga Kau anggap layak duduk di")]

In [ ]:
# Non Prayer content
# Same with prayer content
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Jemaat yang dikasihi Tuhan. Bersama ")]

In [ ]:
# priest's confession
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Untuk gereja-gereja yang sudah terbiasa")]

In [ ]:
# Prayer ( There is "PF" )
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Memanjatkan doa syafaat,")]

## Search format of The Sermon in text

In [ ]:
# Ayat memiliki pola (ReGex bisa diimplementasikan untuk mendeteksi ini)
# isi ayat.....(kitabnya)

In [ ]:
# There is '"..."' and Verse (Lukas 10:1)
df_data_paragraph.loc[df_data_paragraph.VALUE.str.contains("Yesaya 11:10")]

In [ ]:
# The verse fragments in the sermon need to be checked manually

## Data Labeling 

In [ ]:
simple_teks = df_data_paragraph.iloc[0:4][['VALUE']].copy()
song_teks = df_data_paragraph.loc[[1021, 1272, 1219, 1007,1008]][['VALUE']].copy()

## Data cleaning (Test)

In [ ]:
# casefolding
simple_teks['cleaned_data'] = simple_teks['VALUE'].str.lower()
song_teks['cleaned_data'] = song_teks['VALUE'].str.lower()

In [ ]:
# Remove punctuation, url, \t\b
import re 
import string

simple_teks['cleaned_data'] = simple_teks['cleaned_data'].apply(lambda x: re.sub("https?:\/\/.*[\r\n]*", "", x))
simple_teks['cleaned_data'] = simple_teks['cleaned_data'].apply(lambda x: re.sub(r'[^\w\s]', "", x))
simple_teks['cleaned_data'] = simple_teks['cleaned_data'].apply(lambda x: re.sub('\\s+', ' ', x))
simple_teks['cleaned_data'] = simple_teks['cleaned_data'].apply(lambda x: '' .join((z for z in x if not z.isdigit())))

song_teks['cleaned_data'] = song_teks['cleaned_data'].apply(lambda x: re.sub("https?:\/\/.*[\r\n]*", "", x))
song_teks['cleaned_data'] = song_teks['cleaned_data'].apply(lambda x: re.sub(r'[^\w\s]', "", x))
song_teks['cleaned_data'] = song_teks['cleaned_data'].apply(lambda x: re.sub('\\s+', ' ', x))
song_teks['cleaned_data'] = song_teks['cleaned_data'].apply(lambda x: '' .join((z for z in x if not z.isdigit())))

In [ ]:
song_teks

In [ ]:
simple_teks

In [ ]:
# # Dumb two sampel
song_teks_clean = song_teks['cleaned_data']
simple_teks_clean = simple_teks['cleaned_data']

simple_teks_clean.to_csv('sample_simple_teks_clean.csv', index=False)
song_teks_clean.to_csv('sample_song_teks_clean.csv', index=False)